In [28]:
%load_ext autoreload
%autoreload 2

import os
import sys
import git

import uproot as ut
import awkward as ak
import numpy as np
import math
import vector
import sympy as sp

import re
from tqdm import tqdm
import timeit

sys.path.append( git.Repo('.', search_parent_directories=True).working_tree_dir )
from utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
tree = Tree(fc.sr.QCD_B_List)

In [32]:
tree['scale']

<Array [0.00436, 0.00436, ... 1.02e-05] type='21943 * float64'>